In [68]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pickle

In [66]:
df = pd.read_parquet('./data/item_factor.parquet/')
len(df['id'].unique())

50632

In [3]:
df.head(3)

,id,features
0,9,"[2.3885365, 2.5284684, 3.907544, 2.88774, -2.4..."
1,19,"[2.3217025, 1.6054373, 1.1222875, 2.641066, -2..."
2,29,"[0.26883575, 3.0826638, 3.1546912, 3.0232868, ..."


In [8]:
df['features'].tolist()

[array([ 2.3885365 ,  2.5284684 ,  3.907544  ,  2.88774   , -2.473387  ,
         1.9493084 , -0.20984317, -1.0745142 , -1.8372407 , -2.2624197 ,
        -0.9962623 , -1.6315931 ,  0.4242047 ,  1.594914  ,  0.38967347,
         3.024071  , -0.36693865, -0.12981522,  0.20312184, -0.9837803 ,
        -0.26256296, -0.5517228 ,  0.11765549, -4.7712073 , -0.4999553 ,
        -1.7759598 , -0.504976  ,  0.14767227,  4.4825053 ,  0.01187057,
        -0.4132869 , -0.66155994,  0.59176207,  1.0236639 ,  0.59982914],
       dtype=float32),
 array([ 2.3217025 ,  1.6054373 ,  1.1222875 ,  2.641066  , -2.822253  ,
         1.9482408 ,  0.06878048,  1.0292188 , -1.8504634 , -2.5898976 ,
        -2.3171618 , -0.32040477,  2.051356  ,  1.618456  , -0.4189205 ,
         0.40321037, -1.7680405 , -1.3496413 ,  1.6439378 , -1.1219621 ,
        -1.3585294 ,  0.22194366,  1.6525875 , -1.426779  , -1.1625118 ,
        -0.63789827, -0.31779832,  1.2209955 ,  3.227092  , -1.0604364 ,
         0.3968594 ,  1.222

In [169]:
model = NearestNeighbors(algorithm="brute", metric='cosine')

In [170]:
model.fit(df['features'].tolist())

NearestNeighbors(algorithm='brute', metric='cosine')

In [171]:
df[df['id']==9]['features'].tolist()

[array([ 2.3885365 ,  2.5284684 ,  3.907544  ,  2.88774   , -2.473387  ,
         1.9493084 , -0.20984317, -1.0745142 , -1.8372407 , -2.2624197 ,
        -0.9962623 , -1.6315931 ,  0.4242047 ,  1.594914  ,  0.38967347,
         3.024071  , -0.36693865, -0.12981522,  0.20312184, -0.9837803 ,
        -0.26256296, -0.5517228 ,  0.11765549, -4.7712073 , -0.4999553 ,
        -1.7759598 , -0.504976  ,  0.14767227,  4.4825053 ,  0.01187057,
        -0.4132869 , -0.66155994,  0.59176207,  1.0236639 ,  0.59982914],
       dtype=float32)]

In [172]:
distance, suggestion = model.kneighbors(df[df['id']==9]['features'].tolist(), n_neighbors=6 )

In [173]:
distance[0]

array([5.9604645e-08, 8.1902325e-02, 1.0431492e-01, 1.0600990e-01,
       1.0651523e-01, 1.0747594e-01], dtype=float32)

In [174]:
books = pd.read_csv('./data/cleaned_books.csv')
indexed_ratings = pd.read_csv('./data/rating_indexed.csv')

result = pd.merge(indexed_ratings, books, on='ISBN')
result["ISBN"].count()

79068

In [175]:
# item_feature
indexed_ratings = pd.read_csv('./data/rating_indexed.csv')

def recommend_book(book_id):

    feature = df[df['id']==id]['features'].tolist()
    distance, suggestion = model.kneighbors(feature, n_neighbors=11)
    ISBN = indexed_ratings[indexed_ratings['item_id'].isin(suggestion[0][1:])]['ISBN'].unique().tolist()
    return ISBN

In [176]:
recommend_book(52199)

['0140280391',
 '0743418174',
 '0446609609',
 '0340592036',
 '0446670014',
 '0380818965',
 '0553440381',
 '0679888268',
 '0679889299',
 '0340590262']

In [168]:
id = 9
ISBN = indexed_ratings[indexed_ratings['item_id']==id]['ISBN'].tolist()[0]
feature = df[df['id']==id]['features'].tolist()
#f = feature.values.reshape(-1,1)
#f[0][0]
feature[0]
distance, suggestion = model.kneighbors(np.reshape(feature.reshape(-1,1)), n_neighbors=10)


AttributeError: 'list' object has no attribute 'reshape'

In [177]:
with open('./model/knn_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [95]:
indexed_ratings = pd.read_csv('./data/rating_indexed.csv')

result = pd.merge(indexed_ratings, books, on='ISBN')

In [106]:
result.head(4)

,Unnamed: 0_x,user_id,ISBN,rating,item_id,Unnamed: 0_y,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,2276,0020960808,10,11965.0,16425,The Natural Superiority of Women,Ashley Montagu,1979,Collier Books,http://images.amazon.com/images/P/0020960808.0...,http://images.amazon.com/images/P/0020960808.0...,http://images.amazon.com/images/P/0020960808.0...
1,1,2276,0030632366,9,4638.0,16458,Outrageous Acts and Everyday Rebellions,Gloria Steinem,1987,Smithmark Pub,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...
2,22696,52917,0030632366,5,4638.0,16458,Outrageous Acts and Everyday Rebellions,Gloria Steinem,1987,Smithmark Pub,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...
3,2,2276,0061030643,8,1011.0,16537,The Associate,Phillip Margolin,2002,HarperTorch,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...


In [108]:
len(result['item_id'].unique())

56097

In [98]:
len(indexed_ratings['item_id'].unique())

64225

In [99]:
len(df['id'].unique())

50632

In [101]:
user_factor = pd.read_parquet('./data/user_factor.parquet/')